# Highlight which streets have been explored 

In [1]:
# import networkx as nx
import osmnx as ox
import pandas as pd
import numpy as np

ox.__version__

'1.8.1'

# Inputs and Definitions

In [2]:
INPUT_ADDRESS = "London, UK"
INPUT_DISTANCE_AROUND_ADDRESS = 1000 # metres
INPUT_GOOGLE_LOCATION_JSON_FILE = "data/LocationHistory/Records.json" # Visit https://takeout.google.com to download a json of location history

THRESHOLD_FOR_VISITED_EDGE = 0.1  # metres. If any google location coordinate is found this close to an OSM edge, it will be marked visited

# Get OSMNX graph of area

In [3]:
# download/model a street network for some city then visualize it
G = ox.graph_from_address(
    INPUT_ADDRESS, network_type="all", dist=INPUT_DISTANCE_AROUND_ADDRESS
)

node_lats = [n[1]["y"] for n in G.nodes(data=True)]
node_lons = [n[1]["x"] for n in G.nodes(data=True)]
gps_bounds = {
    "min_lat": min(node_lats),
    "max_lat": max(node_lats),
    "min_lon": min(node_lons),
    "max_lon": max(node_lons),
}

# Load google location history

In [4]:
df_gps = pd.read_json(INPUT_GOOGLE_LOCATION_JSON_FILE)

# parse lat, lon, and timestamp from the dict inside the locations column
df_gps["lat"] = df_gps["locations"].map(lambda x: x["latitudeE7"])
df_gps["lon"] = df_gps["locations"].map(lambda x: x["longitudeE7"])
df_gps["datetime"] = df_gps["locations"].map(lambda x: x["timestamp"])

# convert lat/lon to decimalized degrees
df_gps["lat"] = df_gps["lat"] / 10.0**7
df_gps["lon"] = df_gps["lon"] / 10.0**7

# drop columns we don't need
df_gps = df_gps.drop(labels=["locations"], axis=1, inplace=False)

# exclude points outside of our min/max bounds
df_gps = df_gps.drop(df_gps[df_gps.lat < gps_bounds["min_lat"]].index)
df_gps = df_gps.drop(df_gps[df_gps.lat > gps_bounds["max_lat"]].index)
df_gps = df_gps.drop(df_gps[df_gps.lon < gps_bounds["min_lon"]].index)
df_gps = df_gps.drop(df_gps[df_gps.lon > gps_bounds["max_lon"]].index)

# save date range for future printing
date_range = "{}-{}".format(df_gps["datetime"].min()[:4], df_gps["datetime"].max()[:4])

# Colour streets that have been visited

In [5]:
# Get the nearest edges per point
nearest_edges, nearest_edges_distances = ox.nearest_edges(
    G, df_gps["lon"].to_list(), df_gps["lat"].to_list(), return_dist=True
)

# Keep only the edges within threshold
visited_indices = np.array(nearest_edges_distances) < THRESHOLD_FOR_VISITED_EDGE
visited_edges = np.array(nearest_edges)[visited_indices]
visited_edges = set([tuple(x) for x in visited_edges[:, 0:2].tolist()])

# Drop unvisited edges
unvisited_edges = [(u,v,k) for u,v,k in G.edges(keys=True) if (u,v) not in visited_edges]
G.remove_edges_from(unvisited_edges)


In [6]:
gdf = ox.graph_to_gdfs(G, nodes=False)


print(f"Streets visited {INPUT_DISTANCE_AROUND_ADDRESS} metres around {INPUT_ADDRESS}, {date_range}")
# gdf.plot() # static
gdf.explore() # interactive

Streets visited 100 metres around London, UK, 2019-2024
